In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import *
from IPython.display import display, HTML, IFrame
%matplotlib inline
from collections import defaultdict
from IPython.display import set_matplotlib_formats
from bs4 import BeautifulSoup
set_matplotlib_formats('png','pdf')


In [2]:
folder = ['compsci-data','jbt-data', 'brands-data-400', 'brands-data-100', 'brands-data-it2', 'jbt-data-it2', 'compsci-data-it2']
MERGE_OTHER = True
ut = 'compsci'
#folder = ['brands-data-it2', 'brands-data-it3', 'brands-data-400', 'brands-data-100']
folder = ['{}-data'.format(ut), '{}-data-it2'.format(ut)]

In [31]:


agg_list = []
full_list = []
test_list = []
for f in folder:
    agg =  pd.read_csv('{}/agg.csv'.format(f))
    agg['type'] = f.split('-')[0]
    agg.set_index('id')
    agg['file'] = f
    agg_list.append(agg)
   
    full = pd.read_csv('{}/full.csv'.format(f))
    full['type'] = f.split('-')[0]
    full.set_index('id')
    full_list.append( full)
    test = pd.read_csv('{}/test.csv'.format(f))

    test = test[test['_hidden'] == False]
    print('{} {}'.format(f,len(test)))
    test['type'] = f.split('-')[0]
    test.set_index('id')
    
    test_list.append (test)
aggregated = pd.concat(agg_list)
if MERGE_OTHER:
    aggregated['label'] = aggregated.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)
    full['label'] = full.apply(lambda row: row['label'] if row['label'] != 'OTHER' else 'NONE', axis=1)


print(aggregated.describe())
full = pd.concat(full_list)
test_questions = pd.concat(test_list)

aggregated = aggregated[aggregated['_unit_state'] == 'finalized']
aggregated['raw_text'] = aggregated.apply(
        lambda row: BeautifulSoup(row['text_html'], "lxml").text.replace(':[OBJECT_A]', '').replace(':[OBJECT_B]', ''),
        axis=1)


def print_info(data):
    avg = pd.merge(data, full, suffixes=['_l', '_r'], on=['id'], how='left')[['id', 'text_readable_r', 'label:confidence', 'label_l', '_golden', 'label_r', '_trust','a','b']].sort_values(['label:confidence', 'label_l' ,'id'])
    renamed_avg = avg.rename(columns={'label_l' : 'assigned_by_crowdflower', 'label_r' : 'proposed_by_annotator'})
    renamed_avg
    return renamed_avg


compsci-data 53
compsci-data-it2 56
           _unit_id  _trusted_judgments  label:confidence  label_gold
count  2.469000e+03         2469.000000       2469.000000         0.0
mean   1.563339e+09            3.073714          0.897640         NaN
std    1.926589e+07            0.316089          0.169504         NaN
min    1.534177e+09            2.000000          0.333300         NaN
25%    1.534177e+09            3.000000          0.697400         NaN
50%    1.576061e+09            3.000000          1.000000         NaN
75%    1.576062e+09            3.000000          1.000000         NaN
max    1.576063e+09            6.000000          1.000000         NaN


In [32]:
from sklearn.model_selection import train_test_split

train_i, test_i = train_test_split(aggregated,stratify=aggregated['label'], test_size=0.2)
print('{} {}'.format(len(train_i), len(test_i)))
print(train_i['_golden'].unique())
print(test_i['_golden'].unique())

1972 493
[False]
[False]


In [33]:
to_remove = test_questions['id'].unique()
print(aggregated.columns)
aggregated = aggregated[~aggregated['id'].isin( to_remove)][['id', 'a', 'b', 'raw_text', 'text_html', 'text_readable', 'type', 'marker']]
aggregated['label_gold'] = ''
aggregated['label_gold_reason'] = ''
aggregated['_golden'] = False
aggregated.to_csv('{}-sentences.csv'.format(ut), index=False)
len(aggregated['id'].unique())

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'label', 'label:confidence', 'a',
       'also_acceptable', 'b', 'expected_label', 'id', 'label_gold', 'marker',
       'raw_text', 'text_html', 'text_readable', 'type', 'file'],
      dtype='object')


2425

In [34]:
testq = test_questions.drop_duplicates(subset=['id'])[['id', 'label_gold', 'text_html', 'text_readable', 'marker', 'a', 'b', 'raw_text', 'type', 'label_gold_reason']]
testq['label_gold'] = testq.apply(lambda row: testq['label_gold'].str.replace('OTHER\n', ''), axis=0)
testq['_golden'] = True
testq.to_csv('{}-test-questions.csv'.format(ut), index=False)
testq

,id,label_gold,text_html,text_readable,marker,a,b,raw_text,type,label_gold_reason,_golden
0,O2373817823,BETTER,That's all going to change now that LESS is be...,That's all going to change now that LESS is be...,True,Ruby,JavaScript,That's all going to change now that LESS is be...,compsci,The implementation of JavaScript is better tha...,True
1,I1409626797,NONE,It is much superior to M4; it can be a useful ...,It is much superior to M4; it can be a useful ...,True,Perl,AWK,It is much superior to M4; it can be a useful ...,compsci,"The sentence is comparative (""It is much super...",True
2,I1282282341,BETTER,"In post #1, we learn the shocking fact that st...","In post #1, we learn the shocking fact that st...",True,Python,Haskell,"In post #1, we learn the shocking fact that st...",compsci,"Haskell is compared to Python, and it's better...",True
3,T3238373942,WORSE,"You can also use <span style=""color: #9A14B2; ...","You can also use *PostgreSQL*, but the API for...",True,MySQL,PostgreSQL,"You can also use PostgreSQL, but the API for t...",compsci,PostgreSQL and MySQL are compared against each...,True
4,K1674852294,OTHER,Many of these are faster and easier to use tha...,Many of these are faster and easier to use tha...,True,Sed,AWK,Many of these are faster and easier to use tha...,compsci,"The sentence is comparative, but it does not c...",True
5,M1988309165,OTHER,Perl is now an arcane language for aging Unix ...,Perl is now an arcane language for aging Unix ...,True,Sed,AWK,Perl is now an arcane language for aging Unix ...,compsci,"The sentence is comparative, but it does not c...",True
6,T3184717899,WORSE\nOTHER,"<span style=""color: #9A14B2; font-weight: bold...",*Windows Vista* will require better performing...,True,Windows XP,Windows Vista,Windows Vista will require better performing h...,compsci,The sentence is comparative; the Operating Sys...,True
7,M1929075528,BETTER,"Ordinarily, I'd do this using a scripting lang...","Ordinarily, I'd do this using a scripting lang...",True,Python,Bash,"Ordinarily, I'd do this using a scripting lang...",compsci,"Python and Bash are compared, and it's said th...",True
8,I1303782459,NONE,"In the early days this was using <span style=""...",In the early days this was using *Smalltalk* a...,False,Java,Smalltalk,In the early days this was using Smalltalk and...,compsci,"OTHER fits for the comparison of ""earlier days...",True
9,T3268875891,WORSE,You're neglecting the fact that everything you...,You're neglecting the fact that everything you...,True,CUDA,OpenCL,You're neglecting the fact that everything you...,compsci,OpenCL and CUDA are compared. OpenCL is worse ...,True


### Quality

Sentences with a confidence <= 0.6. Either all three annotators gave a different answer or the annotators are not trustworthy enough.

In [8]:
import re
from collections import defaultdict
from pprint import pprint
import operator
d = defaultdict(int)
exp = r'[\w\s+*-]+(?=:\[OBJECT_[AB]\])'
s = set()
for i, row in aggregated.iterrows():
    a = (sorted(re.findall(exp, row['text_html'])))
    d['{} vs. {}'.format(a[0],a[1])] += 1
print(len(d))
pprint(sorted(d.items(), key=operator.itemgetter(1)))

42
[('JavaScript vs. Perl', 19),
 ('Floating-point vs. Integer', 20),
 ('Haskell vs. Java', 20),
 ('AWK vs. Perl', 23),
 ('Java vs. Lisp', 25),
 ('MySQL vs. SQLite', 27),
 ('Haskell vs. Python', 29),
 ('Bash vs. Python', 35),
 ('Java vs. Objective-C', 38),
 ('Apple TV vs. iOS', 40),
 ('PHP vs. Ruby', 41),
 ('AWK vs. Sed', 43),
 ('JavaScript vs. Ruby', 44),
 ('Oracle vs. PostgreSQL', 44),
 ('Lisp vs. Python', 47),
 ('PHP vs. Python', 47),
 ('Perl vs. Ruby', 47),
 ('Bluetooth vs. Ethernet', 47),
 ('Windows 98 vs. Windows XP', 48),
 ('Java vs. Perl', 58),
 ('CUDA vs. OpenCL', 59),
 ('Groovy vs. Java', 59),
 ('Windows 8 vs. Windows XP', 60),
 ('Objective-C vs. Swift', 63),
 ('Windows Vista vs. Windows XP', 66),
 ('PHP vs. Perl', 66),
 ('JavaScript vs. Python', 67),
 ('Perl vs. Python', 69),
 ('MySQL vs. PostgreSQL', 73),
 ('Java vs. Ruby', 73),
 ('Java vs. PHP', 76),
 ('Direct3D vs. OpenGL', 78),
 ('JavaScript vs. PHP', 81),
 ('MySQL vs. Oracle', 82),
 ('Java vs. Scala', 83),
 ('Java vs. S

## Results

**Label Distribution**

In [10]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

def show_dist(labels):

    plt.figure(figsize=plt.figaspect(1))
    plt.pie(
        labels.value_counts().values,
        labels=labels.value_counts().keys().tolist(),
        radius=2,
        autopct=make_autopct(labels.value_counts().values))
    plt.show()


**All sentences**

In [36]:
compsci = pd.read_csv('compsci-sentences.csv')
brands = pd.read_csv('brands-sentences.csv')
combi = pd.concat([compsci,brands])
print('{}+{}={} ({})'.format(len(compsci), len(brands),len(combi), (len(compsci)+len(brands))))
compsci[compsci.duplicated('id', keep=False)].sort_values('id')

2425+2522=4947 (4947)


,id,a,b,raw_text,text_html,text_readable,type,marker,label_gold,label_gold_reason,_golden


In [40]:
test_questions[~test_questions.duplicated('id')]

,_id,_pct_missed,_judgments,_hidden,_contention,_pct_contested,_gold_pool,_review_state,_label,label_gold,...,id,text_html,text_readable,expected_label,a,b,marker,raw_text,also_acceptable,type
0,1534179544,0.06,31,False,NaN,0.0000,NaN,NaN,NaN,BETTER,...,O2373817823,That's all going to change now that LESS is be...,That's all going to change now that LESS is be...,O,Ruby,JavaScript,True,That's all going to change now that LESS is be...,NaN,compsci
1,1534179721,0.15,41,False,NaN,0.0244,NaN,NaN,NaN,OTHER\nNONE,...,I1409626797,It is much superior to M4; it can be a useful ...,It is much superior to M4; it can be a useful ...,NaN,Perl,AWK,True,It is much superior to M4; it can be a useful ...,NaN,compsci
2,1534179815,0.21,29,False,NaN,0.0000,NaN,NaN,NaN,BETTER,...,I1282282341,"In post #1, we learn the shocking fact that st...","In post #1, we learn the shocking fact that st...",NaN,Python,Haskell,True,"In post #1, we learn the shocking fact that st...",NaN,compsci
3,1534179888,0.14,22,False,NaN,0.0000,NaN,NaN,NaN,WORSE,...,T3238373942,"You can also use <span style=""color: #9A14B2; ...","You can also use *PostgreSQL*, but the API for...",NaN,MySQL,PostgreSQL,True,"You can also use PostgreSQL, but the API for t...",NaN,compsci
4,1534180048,0.33,43,False,he compares them both to completely different ...,0.0233,NaN,passed,NaN,OTHER,...,K1674852294,Many of these are faster and easier to use tha...,Many of these are faster and easier to use tha...,NaN,Sed,AWK,True,Many of these are faster and easier to use tha...,NaN,compsci
5,1534180167,0.32,38,False,NaN,0.0000,NaN,NaN,NaN,OTHER,...,M1988309165,Perl is now an arcane language for aging Unix ...,Perl is now an arcane language for aging Unix ...,NaN,Sed,AWK,True,Perl is now an arcane language for aging Unix ...,NaN,compsci
6,1534180347,0.48,48,False,unfair,0.0208,NaN,passed,NaN,WORSE\nOTHER,...,T3184717899,"<span style=""color: #9A14B2; font-weight: bold...",*Windows Vista* will require better performing...,NaN,Windows XP,Windows Vista,True,Windows Vista will require better performing h...,NaN,compsci
7,1534180420,0.35,43,False,please check it again and fix my accuracy,0.0233,NaN,passed,NaN,BETTER,...,M1929075528,"Ordinarily, I'd do this using a scripting lang...","Ordinarily, I'd do this using a scripting lang...",NaN,Python,Bash,True,"Ordinarily, I'd do this using a scripting lang...",NaN,compsci
8,1534180468,0.12,24,False,NaN,0.0000,NaN,NaN,NaN,OTHER\nNONE,...,I1303782459,"In the early days this was using <span style=""...",In the early days this was using *Smalltalk* a...,NaN,Java,Smalltalk,False,In the early days this was using Smalltalk and...,NaN,compsci
9,1534180563,0.23,13,False,NaN,0.0000,NaN,NaN,NaN,WORSE,...,T3268875891,You're neglecting the fact that everything you...,You're neglecting the fact that everything you...,NaN,CUDA,OpenCL,True,You're neglecting the fact that everything you...,NaN,compsci
